In [1]:
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
images = []
for files in os.listdir("/kaggle/input/animals10/raw-img/pecora"):
    if len(images)==125:
        break
    img = cv2.imread(f"/kaggle/input/animals10/raw-img/pecora/{files}")
    img = cv2.resize(img,(256,256))
    images.append(img)

In [3]:
images_t = []
for files in os.listdir("/kaggle/input/tiger-img/tiger"):
    if len(images_t)==125:
        break
    img = cv2.imread(f"/kaggle/input/tiger-img/tiger/{files}")
    img = cv2.resize(img,(256,256))
    images_t.append(img)

In [4]:
label1 = [0 for char in range(125)]
label2 = [1 for char in range(125)]
label = label1+label2

In [5]:
label = np.array(label)
images = np.array(images+images_t)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(images, label, test_size=0.2, random_state=42)

In [7]:
import tensorflow as tf

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(32).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = True

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [10]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x) 
model = tf.keras.Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 32, 32, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,780,481 (56.38 MB)

 Trainable params: 14,780,481 (56.38 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
!pip install tensorboard

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [13]:
import tensorflow as tf
import time

# Set up TensorBoard callback
log_dir = "logs/fit"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

class LogMetricsCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        """ Called at the start of training """
        # Record the start time of the training
        self.start_time = time.time()
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_batch_end(self, batch, logs=None):
        """ Called at the end of each batch """
        step = int(self.model.optimizer.iterations.numpy())  # Convert to integer
        with self.writer.as_default():
            # Record training loss and accuracy
            tf.summary.scalar('training_loss', logs['loss'], step=step)
            tf.summary.scalar('training_accuracy', logs['accuracy'], step=step)

            # Log testing accuracy every 100 batches
            if batch % 100 == 0:
                test_accuracy = self.model.evaluate(test_dataset, verbose=0)[1]  # Get accuracy from test dataset
                tf.summary.scalar('testing_accuracy', test_accuracy, step=step)

            # Calculate and log training time for this batch (optional)
            elapsed_time = time.time() - self.start_time
            tf.summary.scalar('training_time', elapsed_time, step=step)
            self.writer.flush()  # Ensure logs are written

    def on_epoch_end(self, epoch, logs=None):
        """ Called at the end of each epoch """
        elapsed_time = time.time() - self.start_time
        step = int(self.model.optimizer.iterations.numpy())  # Convert to integer
        with self.writer.as_default():
            tf.summary.scalar('training_time', elapsed_time, step=step)
        print(f"Epoch {epoch+1}: Training time so far: {elapsed_time:.2f} seconds")

    def on_train_end(self, logs=None):
        """ Called at the end of training """
        total_training_time = time.time() - self.start_time
        step = int(self.model.optimizer.iterations.numpy())  # Convert to integer
        with self.writer.as_default():
            tf.summary.scalar('total_training_time', total_training_time, step=step)
        print(f"Total training time: {total_training_time:.2f} seconds")
        self.writer.close()  # Close writer at the end of training

# Add this callback to your model's training
history = model.fit(
    train_dataset,  # your training dataset
    epochs=20,
    callbacks=[tensorboard_callback, LogMetricsCallback()]
)


Epoch 1/20


I0000 00:00:1731236211.642012      94 service.cc:145] XLA service 0x7baf9001b450 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731236211.642068      94 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1731236238.463199      94 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_9', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_8', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_7', 4 bytes spill stores, 12 bytes spill loads

I0000 00:00:1731236238.471249      94 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5075 - loss: 2.1553   

I0000 00:00:1731236257.773459      94 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_9', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_6', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_7', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_8', 4 bytes spill stores, 12 bytes spill loads



Epoch 1: Training time so far: 52.64 seconds
7/7 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.5032 - loss: 2.0678
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.5749 - loss: 0.6747Epoch 2: Training time so far: 56.31 seconds
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 548ms/step - accuracy: 0.5770 - loss: 0.6747
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.6074 - loss: 0.6925Epoch 3: Training time so far: 60.05 seconds
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 559ms/step - accuracy: 0.6037 - loss: 0.6896
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.6966 - loss: 0.6488Epoch 4: Training time so far: 63.80 seconds
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 560ms/step - accuracy: 0.6915 - loss: 0.6491
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.7221 - loss: 0.5871Epoch 5: Training time so far: 67.53 seconds
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 558ms/step - accuracy: 0.7213 - loss: 0.5886
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step - accuracy: 0.7301 - loss: 0.5

In [15]:
import shutil
# Create a zip file of the logs directory
shutil.make_archive('/kaggle/working/tensorboard_logs', 'zip', '/kaggle/working/logs')

'/kaggle/working/tensorboard_logs.zip'